In [55]:
import pandas as pd

In [56]:
train_log = pd.read_csv("../data/train_log.csv")
train_label = pd.read_csv("../data/train_label.csv")

test_log = pd.read_csv("../data/test_log.csv")
test_label = pd.read_csv("../data/test_session.csv")

yado = pd.read_csv("../data/yado.csv")

sample_submission = pd.read_csv("../data/sample_submission.csv")

# image_embeddings = pd.read_parquet("../data/image_embeddings.parquet")

In [57]:
# logの数を追加
train_label_rename = train_label.rename(columns={"yad_no": "selected_yad_no"})
session_id_log_num_dict = train_log.groupby("session_id")["yad_no"].count().to_dict()
train_label_rename["session_id_log_num"] = train_label_rename["session_id"].map(
    session_id_log_num_dict
)

In [58]:
# 過去に見たlogのyado_idを追加
train_log_yado_no_sr = train_log.groupby("session_id")["yad_no"].apply(list)
train_label_rename["yad_no_list"] = train_log_yado_no_sr.values

In [59]:
# logが1件のsession_idに限定
train_label_rename_log_cnt_1 = train_label_rename[
    train_label_rename["session_id_log_num"] == 1
]

In [60]:
# listからidに変換
train_label_rename_log_cnt_1["logged_yad_no"] = train_label_rename_log_cnt_1[
    "yad_no_list"
].apply(lambda x: x[0])

/var/folders/4p/kqrm_mh10fn2pjkrxfzxh03c0000gq/T/ipykernel_43394/3627810249.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_label_rename_log_cnt_1["logged_yad_no"] = train_label_rename_log_cnt_1[


In [63]:
train_label_rename_log_cnt_1_with_yado_info = pd.merge(
    train_label_rename_log_cnt_1,
    yado.add_prefix("selected_"),
    on="selected_yad_no",
    how="left",
)

train_label_rename_log_cnt_1_with_yado_info = pd.merge(
    train_label_rename_log_cnt_1_with_yado_info,
    yado.add_prefix("logged_"),
    on="logged_yad_no",
    how="left",
)

In [65]:
def calc_match_rate(df: pd.DataFrame, col: str) -> None:
    match = (
        train_label_rename_log_cnt_1_with_yado_info[f"selected_{col}"]
        == train_label_rename_log_cnt_1_with_yado_info[f"logged_{col}"]
    )

    print(match.count())
    print(match.sum())
    print(match.mean())

In [66]:
calc_match_rate(train_label_rename_log_cnt_1_with_yado_info, "wid_cd")

185386
184344
0.9943792950924017


In [67]:
calc_match_rate(train_label_rename_log_cnt_1_with_yado_info, "ken_cd")

185386
182327
0.9834992933662736


In [68]:
calc_match_rate(train_label_rename_log_cnt_1_with_yado_info, "lrg_cd")

185386
170630
0.9204039139956631


In [69]:
calc_match_rate(train_label_rename_log_cnt_1_with_yado_info, "sml_cd")

185386
140421
0.7574520190305633
